# Import the required libraries <a class="anchor" id="import_libraries"></a>

In [ ]:
%matplotlib widget
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import atomap.api as am
hs.preferences.GUIs.warn_if_guis_are_missing = False
hs.preferences.save()
plt.rcParams['figure.figsize'] = (7,7)

# Muraki model fitting <a class="anchor" id="muraki_model_fitting"></a>
First, we load the data:

In [ ]:
s=hs.load('data.hspy')
intensity_array=np.load('intensity_array.npy')
composition_map=np.load('composition_map.npy')
avg_intensity=np.mean(composition_map[:,:,0],axis=0)
std_dev_intensity=np.std(composition_map[:,:,0],axis=0)
avg_axis=np.mean(composition_map[:,:,1],axis=0)*s.axes_manager[0].scale

nominal_composition=1

According to the Muraki model, the segregation behaviour can be characterized by the segregation coefficient. Therefore, it is necessary to construct the model, select the region of the quantum well (with the interfaces) and find the optimal parameters for the model.

In [ ]:
plt.figure()
plt.plot(np.arange(0,len(avg_intensity)),avg_intensity,'*--')
plt.show()

Constructing the Muraki model for one segregation coefficient:

In [ ]:
from hyperspy.component import Component
class Muraki(Component):
    def __init__(self, parameter_1=1, parameter_2=2, parameter_3=3):
        Component.__init__(self, ('x0', 's', 'N'))
        self.x0.value = nominal_composition
        self.s.value = 0.5
        self.N.value = 20
        self.x0.bmin = 0
        self.x0.bmax = 1
        self.s.bmin = 0
        self.s.bmax = 1
        self.N.bmin = 0
        self.N.bmax = 50
    def function(self, x):
        x0 = self.x0.value
        s = self.s.value
        N = self.N.value
        return np.piecewise(x,[((x >= 1.0) & (x<= N)),x > N],[lambda x : x0*(1.0 -s**x), lambda x: x0*(1 -s**x)*s**(x-N)])

Constructing the Muraki model for two segregation coefficients:

In [ ]:
class Muraki2(Component):
    def __init__(self, parameter_1=1, parameter_2=2, parameter_3=3, parameter_4=4):
        Component.__init__(self, ('x0', 's1', 's2', 'N'))
        self.x0.value = nominal_composition
        self.s1.value = 0.5
        self.s2.value = 0.5
        self.N.value = 20
        self.x0.bmin = 0
        self.x0.bmax = 1
        self.s1.bmin = 0
        self.s1.bmax = 1
        self.s2.bmin = 0
        self.s2.bmax = 1
        self.N.bmin = 0
        self.N.bmax = 50
    def function(self, x):
        x0 = self.x0.value
        s1 = self.s1.value
        s2 = self.s2.value
        N = self.N.value
        return np.piecewise(x,[((x >= 1.0) & (x<= N)),x > N],[lambda x : x0*(1.0 -s1**x), lambda x: x0*(1 -s2**x)*s2**(x-N)])

From the histogram of the average composition plot, we can get the mean value of the barriers and the quantum well (using the two highest peaks of the histogram).

In [ ]:
from scipy.stats import binned_statistic
from scipy.signal import find_peaks
bins=10
count_binned=binned_statistic(avg_intensity,avg_intensity, 'count', bins=bins)
bin_centers=(count_binned[1][1:] + count_binned[1][:-1])/2
mean_binned=binned_statistic(avg_intensity,avg_intensity, 'mean', bins=bins)
peaks, _ = find_peaks(count_binned[0], height=0)
pos_peaks_sorted=peaks[np.argsort(count_binned[0][peaks])]
max1,max2=np.sort(mean_binned[0][pos_peaks_sorted[-2::]])
plt.figure()
plt.hist(avg_intensity,bins=count_binned[1],color='lightblue')
plt.plot(bin_centers,count_binned[0],'*--')
plt.plot(bin_centers[peaks], count_binned[0][peaks], "D")
plt.show()

Then, we select the limits (for the interfaces) considering 5% and the 95% in the interval obtained by the previous mean values (of the quantum well and the barriers).

In [ ]:
b_perc=0.05
t_perc=0.95
blim=b_perc*(max2-max1)+max1
tlim=t_perc*(max2-max1)+max1

plt.figure()
plt.plot(np.arange(0,len(avg_intensity)),avg_intensity,'*')
text1='%d%% : %.3f' % (b_perc*100,blim)
text2='%d%% : %.3f' % (t_perc*100,tlim)
plt.plot(np.arange(0,len(avg_intensity)),[blim]*len(avg_intensity),linestyle='--',color='red',alpha=0.5,label=text1)
plt.plot(np.arange(0,len(avg_intensity)),[tlim]*len(avg_intensity),linestyle='--',color='orange',alpha=0.5,label=text2)
plt.xlabel('Layer')
plt.ylabel('Average composition')
plt.legend()
plt.show()

For the Muraki model fitting, it is selected the mean of the barriers as the limit of the region to fit:

In [ ]:
positions_l=np.where(avg_intensity>max1)
intervals=np.split(positions_l[0], np.where(np.diff(positions_l[0]) != 1)[0]+1)
intervals_len = [len(i) for i in intervals]
muraki_positions=intervals[np.argmax(np.array(intervals_len))]
muraki_signal=avg_intensity[muraki_positions]
std_dev=std_dev_intensity[muraki_positions]
sc=hs.signals.Signal1D(muraki_signal)
sc.axes_manager[0].name = 'Layer'
sc.plot()
plot=plt.gca()
plot.lines[0].set_drawstyle('default')
plot.lines[0].set_linestyle('--')
plot.lines[0].set_marker('v')
plot.lines[0].set_color('blue')

## Fitting (one segregation coefficient): <a class="anchor" id="fitting_1s"></a>

$$x(n) = \Bigg\{ \begin{matrix} x_0(1-S^n): & 1 \leq n \leq N \\  x_0(1-S^N)S^{n-N}: & n>N \end{matrix}$$
(Kükelhan et al.,2019)

In [ ]:
muraki_model = sc.create_model()
muraki = Muraki()
muraki_model.append(muraki)
muraki_model.fit()
muraki_model.print_current_values()

Plotting the fitted model and the input data:
>For the **`r2_parameter`** calculation is not considered the layer 0.

In [ ]:
from sklearn.metrics import r2_score
def f(x,x0,s,N):
    return np.piecewise(x,[((x >= 1.0) & (x<= N)),x > N],[lambda x : x0*(1.0 -s**x), lambda x: x0*(1 -s**x)*s**(x-N)])
x=np.arange(0,len(sc.data),dtype=float)
y_pred=f(x,muraki.x0.value,muraki.s.value,muraki.N.value)
r2_parameter=r2_score(sc.data[1::], y_pred[1::])

from matplotlib.offsetbox import AnchoredText
plt.figure()
plt.errorbar(x,sc.data, yerr=std_dev, ecolor='skyblue',marker=".",fmt=':', capsize=3,alpha=0.75)
plt.fill_between(x,sc.data-std_dev,sc.data+std_dev, alpha=.25)
plt.plot(x,y_pred,color='red')
plt.xlabel('Layer')
plt.ylabel('Average Composition')
plt.minorticks_on()
plot=plt.gca()
label='$R^2 = $'+str(np.round(r2_parameter,3))
at = AnchoredText(label, prop=dict(size=10), frameon=True, loc='upper right')
at.patch.set(edgecolor='lightgray')
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.1")
plot.add_artist(at)
plt.show()

plt.figure()
plt.plot(avg_axis[np.arange(0,len(avg_intensity))],avg_intensity,'*--')
plt.plot(avg_axis[x.astype(int)+muraki_positions[0]],y_pred,'-',color='red')
plt.xlabel('Position [nm]')
plt.ylabel('Average Composition')
plt.minorticks_on()
plt.show()

## Fitting (two segregation coefficients):: <a class="anchor" id="fitting_2s"></a>

$$x(n) = \Bigg\{ \begin{matrix} x_0(1-S^n_{l}): & 1 \leq n \leq N \\  x_0(1-S^N_l)S^{n-N}_u: & n>N \end{matrix}$$
(Kükelhan et al.,2019)

In [ ]:
muraki_model = sc.create_model()
muraki = Muraki2()
muraki_model.append(muraki)
muraki_model.fit()
muraki_model.print_current_values()

Plotting the fitted model and the input data:
>For the **`r2_parameter`** calculation is not considered the layer 0.

In [ ]:
from sklearn.metrics import r2_score
def f(x,x0,s1,s2,N):
    return np.piecewise(x,[((x >= 1.0) & (x<= N)),x > N],[lambda x : x0*(1.0 -s1**x), lambda x: x0*(1 -s2**x)*s2**(x-N)])
x=np.arange(0,len(sc.data),dtype=float)
y_pred=f(x,muraki.x0.value,muraki.s1.value,muraki.s2.value,muraki.N.value)
r2_parameter=r2_score(sc.data[1::], y_pred[1::])

plt.figure()
plt.errorbar(x,sc.data, yerr=std_dev, ecolor='skyblue',marker=".",fmt=':', capsize=3,alpha=0.75)
plt.fill_between(x,sc.data-std_dev,sc.data+std_dev, alpha=.25)
plt.plot(x,y_pred,color='red')
plt.xlabel('Layer')
plt.ylabel('Average Composition')
plt.minorticks_on()
plot=plt.gca()
label='$R^2 = $'+str(np.round(r2_parameter,3))
at = AnchoredText(label, prop=dict(size=10), frameon=True, loc='upper right')
at.patch.set(edgecolor='lightgray')
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.1")
plot.add_artist(at)
plt.show()

plt.figure()
plt.plot(avg_axis[np.arange(0,len(avg_intensity))],avg_intensity,'*--')
plt.plot(avg_axis[x.astype(int)+muraki_positions[0]],y_pred,'-',color='red')
plt.xlabel('Position [nm]')
plt.ylabel('Average Composition')
plt.minorticks_on()
plt.show()

# Topology of the interfaces <a class="anchor" id="topology"></a>

To see how is the topology of the bottom and top interface, we take the interfaces contained in the previous limits.

In [ ]:
positions_int=np.where((avg_intensity>max1) & (avg_intensity<max2))
intervals=np.split(positions_int[0], np.where(np.diff(positions_int[0]) != 1)[0]+1)
intervals_len = [len(i) for i in intervals]
bottom,top=intervals[np.argsort(np.array(intervals_len))[-2::][0]],intervals[np.argsort(np.array(intervals_len))[-2::][1]]
b1,b2=bottom[0],bottom[-1]
t1,t2=top[0],top[-1]

Next, the interfaces are plotted and highlighted in the composition map:

In [ ]:
interfaces=np.zeros_like(composition_map[:,:,0])
interfaces[:]=np.nan
interfaces[:,b1:b2+1]=composition_map[:,b1:b2+1,0]
interfaces[:,t1:t2+1]=composition_map[:,t1:t2+1,0]
non_interface=np.copy(composition_map[:,:,0])
non_interface[:,b1:b2+1]=np.nan
non_interface[:,t1:t2+1]=np.nan

plt.figure()
plt.scatter(composition_map[:,:,1],composition_map[:,:,2],s=20,c=interfaces,cmap='jet')
plt.colorbar()
plt.scatter(composition_map[:,:,1],composition_map[:,:,2],s=20,c=non_interface,cmap='gray')
plt.axis('scaled')
plt.axis('off')
ax=plt.gca()
ax.set_ylim(ax.get_ylim()[::-1]) 
ax.xaxis.tick_top() 
ax.yaxis.tick_left()
#plt.imshow(s.data,cmap='gray')
plt.tight_layout()
plt.show()

Finally, we average the interfaces along the growth direction to see the topology:

In [ ]:
bottom_interface=np.mean(composition_map[:,b1:b2+1,0],axis=1)
bottom_std=np.std(composition_map[:,b1:b2+1,0],axis=1)
top_interface=np.mean(composition_map[:,t1:t2+1,0],axis=1)
top_std=np.std(composition_map[:,t1:t2+1,0],axis=1)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.errorbar(np.arange(0,len(bottom_interface)),bottom_interface, yerr=bottom_std, ecolor='skyblue',marker='.',fmt=':', capsize=3,alpha=0.75)
ax1.fill_between(np.arange(0,len(bottom_interface)),bottom_interface-bottom_std,bottom_interface+bottom_std, alpha=.25)
ax1.plot(np.arange(0,len(bottom_interface)),[np.mean(bottom_interface)]*len(bottom_interface),linestyle='--',color='red',alpha=0.5)
#ax1.plot(np.arange(0,len(bottom_interface)),bottom_interface,'*--')
ax1.set(xlabel='Layer',ylabel='Average Composition')
ax1.set_title('Bottom Interface')
ax1.minorticks_on()
ax2.errorbar(np.arange(0,len(top_interface)),top_interface, yerr=top_std, ecolor='skyblue',marker='.',fmt=':', capsize=3,alpha=0.75)
ax2.fill_between(np.arange(0,len(top_interface)),top_interface-top_std,top_interface+top_std, alpha=.25)
ax2.plot(np.arange(0,len(top_interface)),[np.mean(top_interface)]*len(top_interface),linestyle='--',color='red',alpha=0.5)
#ax2.plot(np.arange(0,len(top_interface)),top_interface,'*--')
ax2.set(xlabel='Layer',ylabel='Average Composition')
ax2.set_title('Top Interface')
ax2.minorticks_on()
fig.tight_layout()
plt.show()

ssd=np.std(bottom_interface)
# cv=ssd/np.mean(bottom_interface)*100
print('\033[1m' + 'Metrics for the bottom interface:' + '\033[0m')
print('The mean of the plot is '+ str(np.round(np.mean(bottom_interface),4))+' of the composition')
print('The standard deviation is '+ str(np.round(ssd,4))+' of the composition')
ssd=np.std(top_interface)
# cv=ssd/np.mean(top_interface)*100
print('\n\033[1m' + 'Metrics for the top interface:' + '\033[0m')
print('The mean of the plot is '+ str(np.round(np.mean(top_interface),4))+' of the composition')
print('The standard deviation is '+ str(np.round(ssd,4))+' of the composition')

## Topology of the original image <a class="anchor" id="topology_orig"></a>

We can use the same limits (converted to position in nanometers) to find the topology of the interfaces but in the original image.
>The original image does not show the composition, it only shows the intensity normalized to the impinging beam.

In [ ]:
bottom_interface_pos=s.axes_manager[0].scale*np.mean(composition_map[:,b1-1:b2+2,1],axis=0)
top_interface_pos=s.axes_manager[0].scale*np.mean(composition_map[:,t1-1:t2+2,1],axis=0)
left_b=bottom_interface_pos[0]
right_b=bottom_interface_pos[-1]
left_t=top_interface_pos[0]
right_t=top_interface_pos[-1]

x_new=intensity_array[:,:,1]*s.axes_manager[0].scale
y_new=intensity_array[:,:,2]*s.axes_manager[1].scale
intensity_n=np.stack((intensity_array[:,:,0],x_new,y_new),axis=2)
d1,d2,d3=intensity_n.shape

Finding the bottom interface:

In [ ]:
intensity_left=[]
x_left=[]
y_left=[]
for i in range(0,d1):
    i_l=intensity_n[i,:,0][np.where((left_b<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_b))].tolist()
    x_l=intensity_n[i,:,1][np.where((left_b<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_b))].tolist()
    y_l=intensity_n[i,:,2][np.where((left_b<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_b))].tolist()
    intensity_left.append(i_l)
    x_left.append(x_l)
    y_left.append(y_l)
    
intensity_i_array = np.zeros([len(intensity_left),len(max(intensity_left,key = lambda x: len(x)))])
intensity_i_array[:] = np.nan
for i,j in enumerate(intensity_left):
    intensity_i_array[i][0:len(j)] = j

x_values_array = np.zeros([len(x_left),len(max(x_left,key = lambda x: len(x)))])
x_values_array[:] = np.nan
for i,j in enumerate(x_left):
    x_values_array[i][0:len(j)] = j

y_values_array = np.zeros([len(y_left),len(max(y_left,key = lambda x: len(x)))])
y_values_array[:] = np.nan
for i,j in enumerate(y_left):
    y_values_array[i][0:len(j)] = j
    
intensity_array_left=np.stack((intensity_i_array,x_values_array,y_values_array),axis=2)

Same for the top interface:

In [ ]:
intensity_right=[]
x_right=[]
y_right=[]
for i in range(0,d1):
    i_r=intensity_n[i,:,0][np.where((left_t<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_t))].tolist()
    x_r=intensity_n[i,:,1][np.where((left_t<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_t))].tolist()
    y_r=intensity_n[i,:,2][np.where((left_t<intensity_n[i,:,1])&(intensity_n[i,:,1]<right_t))].tolist()
    intensity_right.append(i_r)
    x_right.append(x_r)
    y_right.append(y_r)
    
intensity_i_array = np.zeros([len(intensity_right),len(max(intensity_right,key = lambda x: len(x)))])
intensity_i_array[:] = np.nan
for i,j in enumerate(intensity_right):
    intensity_i_array[i][0:len(j)] = j

x_values_array = np.zeros([len(x_right),len(max(x_right,key = lambda x: len(x)))])
x_values_array[:] = np.nan
for i,j in enumerate(x_right):
    x_values_array[i][0:len(j)] = j

y_values_array = np.zeros([len(y_right),len(max(y_right,key = lambda x: len(x)))])
y_values_array[:] = np.nan
for i,j in enumerate(y_right):
    y_values_array[i][0:len(j)] = j
    
intensity_array_right=np.stack((intensity_i_array,x_values_array,y_values_array),axis=2)

Plotting:

In [ ]:
s.plot(axes_off=True,scalebar=False)
plt.scatter(intensity_array_left[:,:,1],intensity_array_left[:,:,2],s=6,c=intensity_array_left[:,:,0],cmap='jet')
plt.scatter(intensity_array_right[:,:,1],intensity_array_right[:,:,2],s=6,c=intensity_array_right[:,:,0],cmap='jet')
plt.axis('scaled')
plt.axis('off')
ax=plt.gca()
ax.xaxis.tick_top() 
ax.yaxis.tick_left()
plt.tight_layout()
plt.show()

Finally, we average the interfaces along the growth direction to see the topology:

In [ ]:
avg_intensity_b=np.nanmean(intensity_array_left[:,:,0],axis=1)
std_b=np.nanstd(intensity_array_left[:,:,0],axis=1)
avg_axis_b=np.nanmean(intensity_array_left[:,:,2],axis=1)
avg_intensity_t=np.nanmean(intensity_array_right[:,:,0],axis=1)
std_t=np.nanstd(intensity_array_right[:,:,0],axis=1)
avg_axis_t=np.nanmean(intensity_array_right[:,:,2],axis=1)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.errorbar(avg_axis_b,avg_intensity_b, yerr=std_b, ecolor='skyblue',marker='.',fmt=':', capsize=3,alpha=0.75)
ax1.fill_between(avg_axis_b,avg_intensity_b-std_b,avg_intensity_b+std_b, alpha=.25)
ax1.plot(avg_axis_b,[np.mean(avg_intensity_b)]*len(avg_axis_b),linestyle='--',color='red',alpha=0.5)
#ax1.plot(avg_axis_b,avg_intensity_b,'*--')
ax1.set(xlabel='Position [nm]',ylabel='Average intensity')
ax1.set_title('Bottom Interface')
ax1.minorticks_on()
ax2.errorbar(avg_axis_t,avg_intensity_t, yerr=std_t, ecolor='skyblue',marker='.',fmt=':', capsize=3,alpha=0.75)
ax2.fill_between(avg_axis_t,avg_intensity_t-std_t,avg_intensity_t+std_t, alpha=.25)
ax2.plot(avg_axis_t,[np.mean(avg_intensity_t)]*len(avg_axis_t),linestyle='--',color='red',alpha=0.5)
#ax2.plot(avg_axis_t,avg_intensity_t,'*--')
ax2.set(xlabel='Position [nm]',ylabel='Average intensity')
ax2.set_title('Top Interface')
ax2.minorticks_on()
fig.tight_layout()
plt.show()

ssd=np.std(avg_intensity_b)
# cv=ssd/np.mean(avg_intensity_b)*100
print('\033[1m' + 'Metrics for the bottom interface:' + '\033[0m')
print('The mean of the plot is '+ str(np.round(np.mean(avg_intensity_b),4))+' of the intensity')
print('The Standard Deviation is '+ str(np.round(ssd,4))+' of the intensity')
ssd=np.std(avg_intensity_t)
# cv=ssd/np.mean(avg_intensity_t)*100
print('\n\033[1m' + 'Metrics for the top interface:' + '\033[0m')
print('The mean of the plot is '+ str(np.round(np.mean(avg_intensity_t),4))+' of the intensity')
print('The Standard Deviation is '+ str(np.round(ssd,4))+' of the intensity')

---

Kükelhan, P., Firoozabadi, S., Beyer, A., Duschek, L., Fuchs, C., Oelerich, J. O., Stolz, W., &amp; Volz, K. (2019). Segregation at interfaces in (GaIn)As/Ga(AsSb)/(GaIn)As- quantum well heterostructures explored by atomic resolution stem. Journal of Crystal Growth, 524, 125180. https://doi.org/10.1016/j.jcrysgro.2019.125180 